<a href="https://colab.research.google.com/github/VickkiMars/EagleLegal/blob/main/EagleRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain openai faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 39.9 MB/s eta 0:00:00


In [2]:
!cp /content/drive/MyDrive/nigeria_laws_data.zip /content/

In [3]:
!cp /content/drive/MyDrive/extracted_texts.json /content/

In [ ]:
!unzip /content/nigeria_laws_data.zip

Archive:  /content/nigeria_laws_data.zip
   creating: nigeria_laws_data_20251107_112353/
  inflating: nigeria_laws_data_20251107_112353/processing_results.json  
   creating: nigeria_laws_data_20251107_112353/texts/
  inflating: nigeria_laws_data_20251107_112353/texts/_lawsofnigeria_view2.php.txt  
   creating: nigeria_laws_data_20251107_112353/pdfs/
  inflating: nigeria_laws_data_20251107_112353/pdfs/T8.pdf  
  inflating: nigeria_laws_data_20251107_112353/pdfs/U9.pdf  
  inflating: nigeria_laws_data_20251107_112353/pdfs/F7.pdf  
  inflating: nigeria_laws_data_20251107_112353/pdfs/N1.pdf  
  inflating: nigeria_laws_data_20251107_112353/pdfs/N40.pdf  
  inflating: nigeria_laws_data_20251107_112353/pdfs/C21.pdf  
  inflating: nigeria_laws_data_20251107_112353/pdfs/A23.pdf  
  inflating: nigeria_laws_data_20251107_112353/pdfs/N29.pdf  
  inflating: nigeria_laws_data_20251107_112353/pdfs/N65.pdf  
  inflating: nigeria_laws_data_20251107_112353/pdfs/P42.pdf  
  inflating: nigeria_laws_data_

# Retrieve titles for text metadata

In [ ]:
# Install required packages
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium pandas tqdm

import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from tqdm import tqdm
from datetime import datetime

# Set up Chrome options for Colab
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

# Initialize the driver
driver = webdriver.Chrome(options=chrome_options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

def extract_links_with_text_from_page(page_number):
    """Extract URLs and their link text for links matching the two patterns"""
    try:
        url = f"https://placng.org/lawsofnigeria/?page={page_number}"
        print(f"Processing page {page_number}...")
        driver.get(url)

        # Wait for page to load
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )

        links_data = []

        # Find all <a> tags on the page
        all_links = driver.find_elements(By.TAG_NAME, "a")

        for link in all_links:
            href = link.get_attribute('href')
            if href:
                # Check if href matches our two target patterns
                if ("https://placng.org/lawsofnigeria/laws/" in href or
                    "https://placng.org/lawsofnigeria/view2.php?sn=" in href):

                    link_text = link.text.strip()

                    # If no visible text, try other attributes
                    if not link_text:
                        link_text = link.get_attribute('title') or link.get_attribute('innerText') or ''
                        link_text = link_text.strip()

                    links_data.append({
                        'url': href,
                        'link_text': link_text
                    })

        print(f"Page {page_number}: Found {len(links_data)} matching links")
        return links_data

    except Exception as e:
        print(f"Error processing page {page_number}: {e}")
        return []

def create_enhanced_csv():
    """Create CSV with url and link_text columns for all matching links"""
    all_links_data = []
    total_pages = 69

    print(f"🎯 Extracting links matching patterns:")
    print(f"  1. https://placng.org/lawsofnigeria/laws/*")
    print(f"  2. https://placng.org/lawsofnigeria/view2.php?sn=*")
    print(f"📄 Processing {total_pages} pages...")

    for page_num in tqdm(range(1, total_pages + 1), desc="Extracting links"):
        page_links = extract_links_with_text_from_page(page_num)
        all_links_data.extend(page_links)
        time.sleep(0.5)  # Respectful delay

    # Remove duplicates while preserving order
    seen_urls = set()
    unique_links_data = []

    for link in all_links_data:
        if link['url'] not in seen_urls:
            seen_urls.add(link['url'])
            unique_links_data.append(link)

    # Create DataFrame
    df = pd.DataFrame(unique_links_data)

    # Add file type classification
    df['file_type'] = df['url'].apply(
        lambda x: 'PDF' if '/laws/' in x else 'HTML'
    )

    # Reorder columns
    df = df[['url', 'link_text', 'file_type']]

    # Save to CSV
    output_filename = "nigeria_laws_urls_with_link_text.csv"
    df.to_csv(output_filename, index=False)

    print(f"\n✅ CSV created: {output_filename}")
    print(f"📊 Total unique links: {len(df)}")
    print(f"📄 PDF links (/laws/): {len(df[df['file_type'] == 'PDF'])}")
    print(f"🌐 HTML links (view2.php): {len(df[df['file_type'] == 'HTML'])}")
    print(f"📝 Links with non-empty text: {len(df[df['link_text'] != ''])}")

    # Display sample
    print(f"\n🔍 Sample data:")
    print(df.head(10).to_string(index=False))

    return df

# Run the extraction
try:
    df = create_enhanced_csv()
    print(f"\n🎉 Extraction complete! File saved as 'nigeria_laws_urls_with_link_text.csv'")
finally:
    driver.quit()

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,123 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,526 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/m

Extracting links:   0%|          | 0/69 [00:00<?, ?it/s]

Processing page 1...
Page 1: Found 8 matching links


Extracting links:   1%|▏         | 1/69 [00:03<03:26,  3.04s/it]

Processing page 2...
Page 2: Found 8 matching links


...

Processing page 68...
Page 68: Found 8 matching links


Extracting links:  99%|█████████▊| 68/69 [01:22<00:01,  1.15s/it]

Processing page 69...
Page 69: Found 2 matching links


Extracting links: 100%|██████████| 69/69 [01:23<00:00,  1.21s/it]


✅ CSV created: nigeria_laws_urls_with_link_text.csv
📊 Total unique links: 546
📄 PDF links (/laws/): 133
🌐 HTML links (view2.php): 413
📝 Links with non-empty text: 546

🔍 Sample data:
                                             url                                                                       link_text file_type
    https://placng.org/lawsofnigeria/laws/A1.pdf                                   ABUBAKAR TAFAWA BALEWA UNIVERSITY, BAUCHI ACT       PDF
https://placng.org/lawsofnigeria/view2.php?sn=14                                                         ACTS AUTHENTICATION ACT      HTML
https://placng.org/lawsofnigeria/view2.php?sn=15                                        ADMINISTRATION OF JUSTICE COMMISSION ACT      HTML
https://placng.org/lawsofnigeria/view2.php?sn=16                                     ADMINISTRATIVE STAFF COLLEGE OF NIGERIA ACT      HTML
https://placng.org/lawsofnigeria/view2.php?sn=17                                                      ADMIRALTY JURISDICT

In [ ]:
pdf_dir = "/content/nigeria_laws_data_20251107_112353/pdfs"
#texts = "/content/nigeria_laws_data_20251107_112353/texts"

#file = open("/content/nigeria_laws_data_20251107_112353/texts/_lawsofnigeria_view2.php.txt", "r")


In [ ]:
pdf_files = [os.path.join(pdf_dir, f) for f in os.listdir(pdf_dir) if f.lower().endswith('.pdf')]
print(pdf_files)


['/content/nigeria_laws_data_20251107_112353/pdfs/copyright.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/V3.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/I17.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/T23.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/N126.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/INSTITUTEOFPUBLICANALYSTSOFNIGERIAACT.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/V4.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/S8.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/U1.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/O1.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/TRADEMARKSACT.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/N68.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/NATIONALBOUNDARYCOMMISSION_ESTABLISHMENT_ACT.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/A1.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/C37.pdf', '/content/nigeria_laws_data_

In [ ]:
len(pdf_files)

133

In [ ]:
# Step 1: Install dependencies
!sudo apt install tesseract-ocr -y
!pip install pytesseract pdf2image Pillow PyPDF2

# (Optional) Install Poppler for pdf2image
!sudo apt install poppler-utils -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.12).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.


In [ ]:
from pdf2image import convert_from_path
import pytesseract
from PyPDF2 import PdfReader
from PIL import Image
import os
import json

def is_image_pdf(pdf_path):
    """
    Quick heuristic: returns True if the PDF likely contains only images.
    """
    try:
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            text = page.extract_text()
            if text and text.strip():
                return False
        return True
    except:
        return True  # if PyPDF2 can't read, assume image-based

def extract_text_from_pdf(pdf_path):
    """
    Extract text from a PDF — uses PyPDF2 for text-based, pytesseract for image-based.
    Returns a string.
    """
    if is_image_pdf(pdf_path):
        pages = convert_from_path(pdf_path, dpi=300)
        text = ""
        for i, page in enumerate(pages):
            page_text = pytesseract.image_to_string(page, lang="eng")
            text += f"\n\n--- Page {i+1} ---\n\n" + page_text
        return text.strip()
    else:
        reader = PdfReader(pdf_path)
        text = ""
        for i, page in enumerate(reader.pages):
            text += f"\n\n--- Page {i+1} ---\n\n" + (page.extract_text() or "")
        return text.strip()

# Process all PDFs into a list
extracted_texts = [extract_text_from_pdf(p) for p in pdf_files]

# Save extracted_texts into a JSON file
output_json = "extracted_texts.json"
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(extracted_texts, f, ensure_ascii=False, indent=4)

# Copy JSON to Google Drive
drive_path = "/content/drive/MyDrive/extracted_texts.json"  # adjust folder if needed
!cp "{output_json}" "{drive_path}"

print(f"Saved extracted texts to: {drive_path}")


Saved extracted texts to: /content/drive/MyDrive/extracted_texts.json


EXTRACT TEXTS FROM BASE WEBSITE THAT ARE NOT IN PDFS:

In [ ]:
# Install required packages
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium trafilatura requests pandas tqdm

import os
import time
import json
import requests
import pandas as pd
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import trafilatura
from tqdm import tqdm
from datetime import datetime
import re

# Create main output directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
main_output_dir = f"nigeria_laws_texts_{timestamp}"
os.makedirs(main_output_dir, exist_ok=True)
print(f"📁 Output directory: {main_output_dir}")

# Set up Chrome options for Colab
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

# Initialize the driver
driver = webdriver.Chrome(options=chrome_options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

def setup_requests_session():
    """Set up requests session with headers"""
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    })
    return session

def sanitize_filename(filename):
    """Convert link text to safe filename"""
    # Remove invalid characters and limit length
    filename = re.sub(r'[<>:"/\\|?*]', '', filename)
    filename = filename.replace(' ', '_').strip()
    # Limit length to 100 characters
    if len(filename) > 100:
        filename = filename[:100]
    return filename

def extract_text_from_view2_link(session, url, link_text, max_retries=3):
    """Extract text from view2.php link using requests + trafilatura"""
    for attempt in range(max_retries):
        try:
            response = session.get(url, timeout=30)
            response.raise_for_status()

            # Extract text using trafilatura
            text = trafilatura.extract(response.text)
            if text and len(text.strip()) > 100:
                return True, text
            else:
                return False, "No meaningful text content found"

        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(1)
            else:
                return False, f"Failed to extract text: {str(e)}"

def extract_text_with_selenium(url, max_retries=2):
    """Extract text using Selenium as fallback"""
    for attempt in range(max_retries):
        try:
            driver.get(url)
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )

            html_content = driver.page_source
            text = trafilatura.extract(html_content)

            if text and len(text.strip()) > 100:
                return True, text
            else:
                body_text = driver.find_element(By.TAG_NAME, "body").text
                if body_text and len(body_text.strip()) > 100:
                    return True, body_text
                else:
                    return False, "No meaningful text content found"

        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2)
            else:
                return False, f"Failed to extract text: {str(e)}"

def process_view2_links_from_csv(csv_file):
    """Process all view2 links from CSV and save text using link text as filename"""
    # Load CSV
    try:
        df = pd.read_csv(csv_file)
        print(f"📊 Loaded CSV with {len(df)} rows")

        # Find the correct columns
        url_col = None
        text_col = None

        for col in df.columns:
            if 'link' in col.lower() and 'text' not in col.lower():
                url_col = col
            elif 'text' in col.lower():
                text_col = col

        if not url_col:
            url_col = df.columns[0]  # Use first column as fallback

        print(f"🔗 Using URL column: '{url_col}'")
        if text_col:
            print(f"📝 Using link text column: '{text_col}'")
        else:
            print("❌ No link text column found in CSV")
            return []

    except Exception as e:
        print(f"❌ Error loading CSV: {e}")
        return []

    # Filter only view2.php links
    view2_links = df[df[url_col].str.contains('view2.php', na=False)]
    print(f"🎯 Found {len(view2_links)} view2.php links to process")

    if len(view2_links) == 0:
        print("❌ No view2.php links found in the CSV")
        return []

    # Set up session
    session = setup_requests_session()

    # Process each view2 link
    results = []
    successful_extractions = 0

    for index, row in tqdm(view2_links.iterrows(), total=len(view2_links), desc="Extracting texts"):
        url = row[url_col]
        link_text = row[text_col] if pd.notna(row[text_col]) else f"view2_link_{index}"

        result = {
            'url': url,
            'original_link_text': link_text,
            'filename': None,
            'filepath': None,
            'success': False,
            'error': None,
            'timestamp': datetime.now().isoformat()
        }

        try:
            print(f"\n[{index}] Processing: {url}")
            print(f"    Link text: '{link_text}'")

            # Create filename from link text
            if pd.notna(link_text) and link_text.strip():
                filename = sanitize_filename(link_text.strip()) + '.txt'
            else:
                filename = f"view2_link_{index}.txt"

            # Extract text (try requests first, then selenium)
            success, text_or_error = extract_text_from_view2_link(session, url, link_text)

            if not success:
                success, text_or_error = extract_text_with_selenium(url)

            if success:
                # Save text to file
                filepath = os.path.join(main_output_dir, filename)
                with open(filepath, 'w', encoding='utf-8') as f:
                    f.write(text_or_error)

                result['success'] = True
                result['filename'] = filename
                result['filepath'] = filepath
                result['text_length'] = len(text_or_error)
                successful_extractions += 1
                print(f"    ✅ Text saved: {filename} ({len(text_or_error)} characters)")
            else:
                result['error'] = text_or_error
                print(f"    ❌ Extraction failed: {text_or_error}")

        except Exception as e:
            result['error'] = f"Unexpected error: {str(e)}"
            print(f"    ❌ Unexpected error: {str(e)}")

        results.append(result)

        # Respectful delay
        time.sleep(1)

    # Generate summary
    print(f"\n{'='*60}")
    print("📊 EXTRACTION COMPLETE")
    print(f"{'='*60}")
    print(f"Total view2 links processed: {len(view2_links)}")
    print(f"Successful extractions: {successful_extractions}")
    print(f"Failed: {len(view2_links) - successful_extractions}")
    print(f"📁 Texts saved in: {main_output_dir}")

    # Save processing results
    results_data = {
        'metadata': {
            'processing_date': datetime.now().isoformat(),
            'output_directory': main_output_dir,
            'source_csv': csv_file,
            'total_view2_links': len(view2_links),
            'successful_extractions': successful_extractions,
            'failed_extractions': len(view2_links) - successful_extractions
        },
        'results': results
    }

    results_json_path = os.path.join(main_output_dir, 'extraction_results.json')
    with open(results_json_path, 'w', encoding='utf-8') as f:
        json.dump(results_data, f, indent=2, ensure_ascii=False)

    # List created files
    created_files = os.listdir(main_output_dir)
    text_files = [f for f in created_files if f.endswith('.txt')]

    print(f"\n📝 Created {len(text_files)} text files:")
    for i, filename in enumerate(text_files[:10]):  # Show first 10
        print(f"   {i+1}. {filename}")
    if len(text_files) > 10:
        print(f"   ... and {len(text_files) - 10} more files")

    return results_data

# Run the extraction
if __name__ == "__main__":
    try:
        csv_filename = "nigeria_laws_urls_with_link_text.csv"  # Change this to your CSV filename
        if not os.path.exists(csv_filename):
            print(f"❌ CSV file '{csv_filename}' not found!")
            print("💡 Please make sure the CSV file exists in the current directory")
        else:
            results = process_view2_links_from_csv(csv_filename)
    except Exception as e:
        print(f"❌ Script failed: {e}")
    finally:
        driver.quit()
        print(f"\n🎉 Text extraction completed! Files saved in: {main_output_dir}")

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

Extracting texts:   0%|          | 0/413 [00:00<?, ?it/s]


[1] Processing: https://placng.org/lawsofnigeria/view2.php?sn=14
    Link text: 'ACTS AUTHENTICATION ACT'
    ✅ Text saved: ACTS_AUTHENTICATION_ACT.txt (4772 characters)


Extracting texts:   0%|          | 1/413 [00:01<12:33,  1.83s/it]


[2] Processing: https://placng.org/lawsofnigeria/view2.php?sn=15
    Link text: 'ADMINISTRATION OF JUSTICE COMMISSION ACT'
    ✅ Text saved: ADMINISTRATION_OF_JUSTICE_COMMISSION_ACT.txt (7612 characters)


...


[540] Processing: https://placng.org/lawsofnigeria/view2.php?sn=553
    Link text: 'WATER RESOURCES ACT'
    ✅ Text saved: WATER_RESOURCES_ACT.txt (18894 characters)


Extracting texts: 100%|█████████▉| 412/413 [10:28<00:01,  1.54s/it]


[542] Processing: https://placng.org/lawsofnigeria/view2.php?sn=555
    Link text: 'WEST AFRICAN EXAMINATIONS COUNCIL ACT'
    ✅ Text saved: WEST_AFRICAN_EXAMINATIONS_COUNCIL_ACT.txt (31732 characters)


Extracting texts: 100%|██████████| 413/413 [10:29<00:00,  1.52s/it]


📊 EXTRACTION COMPLETE
Total view2 links processed: 413
Successful extractions: 413
Failed: 0
📁 Texts saved in: nigeria_laws_texts_20251109_022035

📝 Created 409 text files:
   1. NATIONAL_EMERGENCY_MANAGEMENT_AGENCY(ESTABLISHMENT,_ETC.)_ACT.txt
   2. MARITIME_ACADEMY_OF_NIGERIA_ACT.txt
   3. NIGERIAN_ACCOUNTING_STANDARDS_BOARD_ACT.txt
   4. FEDERAL_URBAN_MASS_TRANSIT_AGENCY_ACT.txt
   5. NIGERIAN_CUSTOMS_SERVICE_BOARD_ACT.txt
   6. PENAL_CODE_(NORTHERN_STATES)_FEDERAL_PROVISIONS_ACT.txt
   7. NIGERIAN_STEEL_DEVELOPMENT_AUTHORITY_ACT.txt
   8. NNAMDI_AZIKIWE_UNIVERSITY_TEACHING_HOSPITAL_ACT.txt
   9. SCIENTIFlC_AND_INDUSTRIAL_RESEARCH_ACT.txt
   10. TREASURY_BILLS_ACT.txt
   ... and 399 more files

🎉 Text extraction completed! Files saved in: nigeria_laws_texts_20251109_022035


In [ ]:
!zip -r /content/nigeria_laws_texts_20251109_022035.zip /content/nigeria_laws_texts_20251109_022035

  adding: content/nigeria_laws_texts_20251109_022035/ (stored 0%)
  adding: content/nigeria_laws_texts_20251109_022035/NATIONAL_EMERGENCY_MANAGEMENT_AGENCY(ESTABLISHMENT,_ETC.)_ACT.txt (deflated 69%)
  adding: content/nigeria_laws_texts_20251109_022035/MARITIME_ACADEMY_OF_NIGERIA_ACT.txt (deflated 69%)
  adding: content/nigeria_laws_texts_20251109_022035/NIGERIAN_ACCOUNTING_STANDARDS_BOARD_ACT.txt (deflated 70%)
  adding: content/nigeria_laws_texts_20251109_022035/FEDERAL_URBAN_MASS_TRANSIT_AGENCY_ACT.txt (deflated 66%)
  adding: content/nigeria_laws_texts_20251109_022035/NIGERIAN_CUSTOMS_SERVICE_BOARD_ACT.txt (deflated 68%)
  adding: content/nigeria_laws_texts_20251109_022035/PENAL_CODE_(NORTHERN_STATES)_FEDERAL_PROVISIONS_ACT.txt (deflated 74%)
  adding: content/nigeria_laws_texts_20251109_022035/NIGERIAN_STEEL_DEVELOPMENT_AUTHORITY_ACT.txt (deflated 68%)
  adding: content/nigeria_laws_texts_20251109_022035/NNAMDI_AZIKIWE_UNIVERSITY_TEACHING_HOSPITAL_ACT.txt (deflated 63%)
  adding: 

In [ ]:
# Copy to Drive
!cp /content/nigeria_laws_texts_20251109_022035.zip /content/drive/MyDrive/


In [4]:
# copy from drive
!cp /content/drive/MyDrive/nigeria_laws_texts_20251109_022035.zip /content/

In [5]:
!unzip nigeria_laws_texts_20251109_022035.zip

Archive:  nigeria_laws_texts_20251109_022035.zip
   creating: content/nigeria_laws_texts_20251109_022035/
  inflating: content/nigeria_laws_texts_20251109_022035/NATIONAL_EMERGENCY_MANAGEMENT_AGENCY(ESTABLISHMENT,_ETC.)_ACT.txt  
  inflating: content/nigeria_laws_texts_20251109_022035/MARITIME_ACADEMY_OF_NIGERIA_ACT.txt  
  inflating: content/nigeria_laws_texts_20251109_022035/NIGERIAN_ACCOUNTING_STANDARDS_BOARD_ACT.txt  
  inflating: content/nigeria_laws_texts_20251109_022035/FEDERAL_URBAN_MASS_TRANSIT_AGENCY_ACT.txt  
  inflating: content/nigeria_laws_texts_20251109_022035/NIGERIAN_CUSTOMS_SERVICE_BOARD_ACT.txt  
  inflating: content/nigeria_laws_texts_20251109_022035/PENAL_CODE_(NORTHERN_STATES)_FEDERAL_PROVISIONS_ACT.txt  
  inflating: content/nigeria_laws_texts_20251109_022035/NIGERIAN_STEEL_DEVELOPMENT_AUTHORITY_ACT.txt  
  inflating: content/nigeria_laws_texts_20251109_022035/NNAMDI_AZIKIWE_UNIVERSITY_TEACHING_HOSPITAL_ACT.txt  
  inflating: content/nigeria_laws_texts_20251109_0

# MERGE PDFS  (extracted_texts.json) and TEXT FILES TO CREATE GLOBAL INFORMATION SOURCE

In [2]:
import re

def clean_text(text):
    # Replace non-breaking spaces with regular spaces
    text = text.replace('\xa0', ' ')
    # Replace multiple newlines with a single one
    text = re.sub(r'\n+', '\n', text)
    # Remove redundant spaces
    text = re.sub(r' +', ' ', text)
    # Strip leading/trailing whitespace
    return text.strip()

In [4]:
import json

with open("extracted_texts.json", "r") as file:
  pdf_data = json.load(file)

In [5]:
pdf_data = [clean_text(i) for i in pdf_data]

extract all data from txt directory into a list

In [13]:
import os
base_dir = "/content/nigeria_laws_texts_20251109_022035"
files = os.listdir(base_dir)
txt_data = []
for file in files:
  data = open(base_dir+'/'+file, "r")
  txt_data.append(data.read())

In [21]:
l1 = [1,2,3]
l2 = [4,5,6]
l3 = l1+l2

In [22]:
print(l3)

[1, 2, 3, 4, 5, 6]


In [23]:
global_list = pdf_data + txt_data

# Save to faiss index

In [24]:
!pip install faiss-cpu sentence-transformers langchain-text-splitters

In [34]:
import json

with open("acts.json", "w", encoding="utf-8") as f:
    json.dump(global_list, f, ensure_ascii=False, indent=2)


In [35]:
!ls -lh acts.json

-rw-r--r-- 1 root root 29M Nov 12 23:33 acts.json


In [27]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from google.colab import ai

# -----------------------------
# 1. Documents and embeddings
# -----------------------------
documents = global_list  # list of PDF-extracted texts
encoder = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
doc_embeddings = encoder.encode(documents, convert_to_numpy=True)

index = faiss.IndexFlatIP(doc_embeddings.shape[1])
index.add(doc_embeddings)

# -----------------------------
# 2. Dense retrieval
# -----------------------------
def retrieve(query, top_k=5):
    query_emb = encoder.encode([query], convert_to_numpy=True)
    scores, idxs = index.search(query_emb, top_k)
    return [documents[i] for i in idxs[0]]

# -----------------------------
# 3. Multi-hop RAG with LLM reasoning
# -----------------------------
def rag_multi_hop(query, num_hops=2, top_k=3):
    retrieved_docs = []
    current_query = query
    for _ in range(num_hops):
        top_docs = retrieve(current_query, top_k)
        retrieved_docs.extend(top_docs)
        current_query += " " + " ".join(top_docs)  # query expansion

    context = "\n\n".join(retrieved_docs)
    prompt = f"Question: {query}\n\nContext:\n{context}\n\nThink step by step and provide an answer. If you cannot get an answer from the context, respond with 'I don't know' "
    return ai.generate_text(prompt)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [28]:
faiss.write_index(index, "EagleLegal.faiss")


In [29]:
query = "What are the functions of the Vice-Chancellor of Abubakar Tafawa Balewa university"
answer = rag_multi_hop(query, num_hops=2, top_k=3)
print(answer)

The functions of the Vice-Chancellor of Abubakar Tafawa Balewa University, as outlined in the provided text, are as follows:

**General Functions:**
*   **Directing University Activities:** The Vice-Chancellor has the general function of directing the activities of the University (Section 9(2)).
*   **Chief Executive and Academic Officer:** Serves as the Chief Executive and Academic Officer of the University (Section 9(2)).
*   **Ex Officio Chairman of the Senate:** The Vice-Chancellor is the ex officio Chairman of the Senate (Section 9(2) and Third Schedule, Article 3(2)).
*   **Precedence:** Takes precedence before all other members of the University, except the Pro-Chancellor and any other person for the time being acting as Chairman of the Council (Section 9(1)).

**Specific Functions/Powers:**
*   **Staff Discipline (General):** Sets up Investigation Panels to determine if a prima facie case has been established against staff members for misconduct or other disciplinary reasons (S

In [30]:
!cp EagleLegal.faiss /content/drive/MyDrive/

In [31]:
!ls -lh EagleLegal.faiss

-rw-r--r-- 1 root root 1.6M Nov 12 23:28 EagleLegal.faiss
